In [ ]:
!pip install optuna
!pip install -U xgboost

In [7]:
# =====================================================
# XGBoost Model with Optuna Hyperparameter Optimization
# =====================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from xgboost.callback import EarlyStopping
from xgboost import XGBClassifier
from scipy.stats import mode
import optuna

# ======================
# 1. Load Data
# ======================
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

# Drop id if exists
if 'id' in train.columns:
    train.drop('id', axis=1, inplace=True)
if 'id' in test.columns:
    test_id = test['id']
    test.drop('id', axis=1, inplace=True)
else:
    test_id = pd.Series(range(len(test)), name='id')

# Split target and features
X_train = train.drop('WeightCategory', axis=1)
y_train = train['WeightCategory']
X_test = test

# ======================
# 2. One-Hot Encode categorical columns
# ======================
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()
numeric_cols = X_train.select_dtypes(exclude='object').columns.tolist()

ohe = OneHotEncoder(drop=None, sparse_output=False, handle_unknown='ignore')

X_train_cat_encoded = ohe.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = ohe.transform(X_test[categorical_cols])

X_train_cat_encoded = pd.DataFrame(
    X_train_cat_encoded,
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train.index
)
X_test_cat_encoded = pd.DataFrame(
    X_test_cat_encoded,
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test.index
)

X_train_encoded = pd.concat([X_train[numeric_cols], X_train_cat_encoded], axis=1)
X_test_encoded = pd.concat([X_test[numeric_cols], X_test_cat_encoded], axis=1)

# ======================
# 3. Encode target labels
# ======================
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

print(f"\nEncoded Train Shape: {X_train_encoded.shape}")
print(f"Encoded Test Shape: {X_test_encoded.shape}")

# ======================
# 4. Define Optuna Objective
# ======================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.5, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "subsample": trial.suggest_float("subsample", 0.9, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.9, 1.0),
        "random_state": 42,
        "use_label_encoder": False,
        "eval_metric": "mlogloss",
        "n_jobs": -1,
    }

    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    model = XGBClassifier(**params)

    scores = cross_val_score(model, X_train_encoded, y_train_encoded,
                             cv=kf, scoring=make_scorer(accuracy_score))
    return scores.mean()

# ======================
# 5. Run Optuna Study
# ======================
print("\n Running Optuna optimization (this may take a few minutes)...")
opt = optuna.create_study(direction="maximize")
opt.optimize(objective, n_trials=130, show_progress_bar=True)

print("\n===== Optuna Optimization Results =====")
print(f"Best CV Accuracy: {opt.best_value:.4f}")
print("Best Parameters Found:")
for k, v in opt.best_params.items():
    print(f"  {k}: {v}")

# ======================
# 6. Use Best Parameters
# ======================
best_params = opt.best_params
best_params.update({
    "random_state": 42,
    "use_label_encoder": False,
    "eval_metric": "mlogloss",
    "n_jobs": -1
})

print("\n===== Using Best Parameters for Final Training =====")
print(best_params)

# ======================
# 7. 5-Fold Cross Validation Training
# ======================
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
test_preds = np.zeros((X_test_encoded.shape[0], kf.n_splits))
val_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_encoded, y_train_encoded)):
    print(f"\n===== Fold {fold + 1} =====")
    X_tr, X_val = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_tr, y_val = y_train_encoded[train_idx], y_train_encoded[val_idx]

    model = XGBClassifier(**best_params)
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)

    val_pred = model.predict(X_val)
    acc = accuracy_score(y_val, val_pred)
    val_scores.append(acc)
    print(f"Validation Accuracy: {acc:.4f}")

    test_preds[:, fold] = model.predict(X_test_encoded)

# ======================
# 8. Ensemble Test Predictions
# ======================
if len(np.unique(y_train_encoded)) > 2:
    final_preds = mode(test_preds, axis=1)[0].astype(int).ravel()
else:
    final_preds = np.round(test_preds.mean(axis=1)).astype(int)

print("\n===== Final Results =====")
print(f"Mean CV Accuracy: {np.mean(val_scores):.4f}")
print(f"Std Dev: {np.std(val_scores):.4f}")

# Decode predictions
final_labels = label_encoder.inverse_transform(final_preds)

# ======================
# 9. Create Submission File
# ======================
submission = pd.DataFrame({
    'id': test_id,
    'WeightCategory': final_labels
})

print("\n Prediction Preview:")
print(submission.head())
print(f"\nTotal rows: {len(submission)}")

submission.to_csv("predictions.csv", index=False)
print("\n Saved 'predictions.csv' — ready for Kaggle upload!")


[I 2025-10-27 04:05:44,254] A new study created in memory with name: no-name-1ccc1eab-2e2e-4e16-b23b-d27266f44950


Train Shape: (15533, 18)
Test Shape: (5225, 17)

Encoded Train Shape: (15533, 30)
Encoded Test Shape: (5225, 30)

 Running Optuna optimization (this may take a few minutes)...


  0%|          | 0/130 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [04:05:44] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [04:05:56] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [04:06:07] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-10-27 04:06:18,859] Trial 0 finished with value: 0.8797400029181971 and parameters: {'n_estimators': 929, 'learning_rate': 0.006244009626830618, 'max_depth': 3, 'subsample': 0.942464485958811, 'colsample_bytree': 0.967061695227497}. Best is trial 0 with value: 0.8797400029181971.


...

Validation Accuracy: 0.9034

===== Fold 4 =====


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:29:47] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Validation Accuracy: 0.9076

===== Fold 5 =====


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:30:31] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Validation Accuracy: 0.9053

===== Final Results =====
Mean CV Accuracy: 0.9046
Std Dev: 0.0017

 Prediction Preview:
      id       WeightCategory
0  15533     Obesity_Type_III
1  15534   Overweight_Level_I
2  15535  Overweight_Level_II
3  15536      Obesity_Type_II
4  15537        Normal_Weight

Total rows: 5225

 Saved 'predictions.csv' — ready for Kaggle upload!
